# Malaria Detection using Deep Learning


https://www.kaggle.com/iarunava/cell-images-for-detecting-malaria

### Importing necessary libraries

In [1]:
pwd

'/home/wsuser/work'

In [2]:
import tensorflow #open source used for both ML and DL for computation
from tensorflow.keras.models import Sequential #it is a plain stack of layers
from tensorflow.keras.layers import Dense#Dense layer is the regular deeply connected neural network layer
from tensorflow.keras.layers import Flatten#used fot flattening the input or change the dimension
from tensorflow.keras.layers import Conv2D#Convolutional layer
from tensorflow.keras.layers import MaxPool2D# used for downsampling of the input
from tensorflow.keras.layers import Dropout #used for preventing overfitting of model
from tensorflow.keras.preprocessing.image import ImageDataGenerator# to artificially expand the size of a training dataset
import numpy as np#used for numerical analysis

In [3]:
img_width=64 #setting default image width size 
img_height=64 #setting default image height size

### Image Data Agumentation

In [4]:
datagen=ImageDataGenerator(rescale=1/255.0,validation_split=0.2) #Image data augmentation

In [5]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.

if os.environ.get('RUNTIME_ENV_LOCATION_TYPE') == 'external':
    endpoint_f6799ad66c264fd895b3f9a54eab8f7f = 'https://s3.us.cloud-object-storage.appdomain.cloud'
else:
    endpoint_f6799ad66c264fd895b3f9a54eab8f7f = 'https://s3.private.us.cloud-object-storage.appdomain.cloud'

client_f6799ad66c264fd895b3f9a54eab8f7f = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='lOYok4uUDut4GBrUHqmCo0AEf2Yimdy1llHbjvoMtLsF',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url=endpoint_f6799ad66c264fd895b3f9a54eab8f7f)

streaming_body_1 = client_f6799ad66c264fd895b3f9a54eab8f7f.get_object(Bucket='maleriadetection-donotdelete-pr-kacwcpgkgw68ku', Key='cell_images.zip')['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/


In [6]:
from io import BytesIO
import zipfile
unzip=zipfile.ZipFile(BytesIO(streaming_body_1.read()),'r')
file_paths=unzip.namelist()
for path in file_paths:
    unzip.extract(path)

In [7]:
pwd

'/home/wsuser/work'

In [8]:
import os
filenames=os.listdir('/home/wsuser/work/cell_images')

In [9]:
train_data_generator=datagen.flow_from_directory(directory='/home/wsuser/work/cell_images',
                                                target_size=(img_width,img_height),class_mode="binary",
                                                batch_size=16,subset="training")#applying image data agumentation to training data

Found 22048 images belonging to 2 classes.


In [10]:
validation_data_generator=datagen.flow_from_directory(directory='/home/wsuser/work/cell_images',
                                                target_size=(img_width,img_height),class_mode="binary",
                                                batch_size=16,subset="validation")#applying image data agumentation to testing data

Found 5510 images belonging to 2 classes.


In [11]:
train_data_generator.class_indices

{'Parasitized': 0, 'Uninfected': 1}

In [12]:
train_data_generator.labels #checking the labels

array([0, 0, 0, ..., 1, 1, 1], dtype=int32)

### Creating the Model

In [13]:
model= Sequential() #our model

model.add(Conv2D(16,(3,3),input_shape=(img_width,img_height,3),activation="relu"))#convolutional layer
model.add(MaxPool2D(2,2)) # Downsamples the input
model.add(Dropout(0.2)) #drop input randomly for preventing from overfitting

model.add(Conv2D(32,(3,3),activation="relu"))
model.add(MaxPool2D(2,2))
model.add(Dropout(0.3))

model.add(Flatten())#flatten the dimension of the image
model.add(Dense(64,activation="relu"))#deeply connected neural network layer
model.add(Dropout(0.5))

model.add(Dense(1,activation="sigmoid")) #output layer with one neuron


In [14]:
model.summary() #summary of our neural network

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 62, 62, 16)        448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 31, 31, 16)        0         
_________________________________________________________________
dropout (Dropout)            (None, 31, 31, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 29, 29, 32)        4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 14, 14, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 6272)              0

### Compiling the model

In [15]:
#compile modelb
model.compile(optimizer='adam', loss='binary_crossentropy', metrics = ['accuracy'])

### Fitting the model

In [16]:
history = model.fit_generator(generator=train_data_generator,
                              steps_per_epoch = len(train_data_generator),
                              epochs = 5,
                              validation_data = validation_data_generator,
                              validation_steps = len(validation_data_generator))

Epoch 1/5


/opt/conda/envs/Python-3.8-main/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1839: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


1378/1378 [==============================] - 121s 88ms/step - loss: 0.6279 - accuracy: 0.6350 - val_loss: 0.2336 - val_accuracy: 0.9274
Epoch 2/5
1378/1378 [==============================] - 108s 78ms/step - loss: 0.2268 - accuracy: 0.9196 - val_loss: 0.2143 - val_accuracy: 0.9368
Epoch 3/5
1378/1378 [==============================] - 107s 78ms/step - loss: 0.2054 - accuracy: 0.9323 - val_loss: 0.1842 - val_accuracy: 0.9379
Epoch 4/5
1378/1378 [==============================] - 106s 77ms/step - loss: 0.1883 - accuracy: 0.9396 - val_loss: 0.1722 - val_accuracy: 0.9410
Epoch 5/5
1378/1378 [==============================] - 107s 78ms/step - loss: 0.1682 - accuracy: 0.9453 - val_loss: 0.1635 - val_accuracy: 0.9428


### Saving our model

In [17]:
from tensorflow.keras.models import load_model

model.save('maleria.h5')

In [18]:
!tar -zcvf image-classification-model_new.tgz maleria.h5

maleria.h5


In [19]:
ls -1

cell_images/
image-classification-model_new.tgz
maleria.h5


In [20]:
!pip install watson-machine-learning-client --upgrade

     |████████████████████████████████| 538 kB 17.0 MB/s eta 0:00:01


In [21]:
from ibm_watson_machine_learning import APIClient
wml_credentials={
    "url":"https://us-south.ml.cloud.ibm.com",
    "apikey":"PJZUbnrFtGA5cAk57HCRHogyZvL6wbZsDiluFINxylY2"
}
client = APIClient(wml_credentials)


In [22]:
client = APIClient(wml_credentials)


In [23]:
def guid_from_space_name(client,space_name):
    space=client.spaces.get_details()
    return(next(item for item in space['resources'] if item['entity']["name"]==space_name)['metadata']['id'])

In [24]:
space_uid=guid_from_space_name(client,'maleria_deploy')
print("Space UID="+space_uid)

Space UID=9bcc5b10-a43a-439f-85c4-64942ba6c87f


In [25]:
client.set.default_space(space_uid)

'SUCCESS'

In [26]:
client.software_specifications.list()

-----------------------------  ------------------------------------  ----
NAME                           ASSET_ID                              TYPE
default_py3.6                  0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
pytorch-onnx_1.3-py3.7-edt     069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6        09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12     09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
ai-function_0.1-py3.6          0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                     0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod   1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6              10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl      111e41b3-de2d-5422-a4d6-bf776828c4b7  base
scikit-learn_0.22-py3.6        154010fa-5b3b-4ac1-82af-4d5ee5abbc85  base
default_r3.6                   1b70aec3-ab34-4b87-8aa0-a4a3c8296a36  base
pytorch-onnx_1.3-py3.6         1bc6029

In [27]:
#software_spec_uid=client.software_specifications.get_uid_by_name("tensorflow_1.15-py3.6")
software_spec_uid=client.software_specifications.get_uid_by_name("tensorflow_2.4-py3.7")
software_spec_uid

'65e171d7-72d1-55d9-8ebb-f813d620c9bb'

In [28]:
# import tensorflow
# tensorflow.__version__

In [29]:
# !pip install keras==2.2.4

In [30]:
model_details = client.repository.store_model(model='image-classification-model_new.tgz',meta_props={
    client.repository.ModelMetaNames.NAME:"CNN",
    client.repository.ModelMetaNames.TYPE:"keras_2.2.4",
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID:software_spec_uid
})
model_id = client.repository.get_model_uid(model_details)

Note: Warnings!! :  Model type keras_2.2.4 is deprecated. We recommend you use a supported model type. See Supported Frameworks https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/pm_service_supported_frameworks.html


In [31]:
model_id

'e10d376d-ed76-4642-9dc6-0321c3f2e733'

### Predicting our results

In [32]:
client.repository.download(model_id,'my_model7.tar.gz')

Successfully saved model content to file: 'my_model7.tar.gz'


'/home/wsuser/work/my_model7.tar.gz'

In [33]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
model = load_model('maleria.h5') #loading our trained model

In [34]:
y_pred = model.predict(validation_data_generator) #performing prediction to the test data

In [35]:
import numpy as np
y_pred = np.argmax(y_pred, axis=1)#Returns the indices of the maximum values along an axis
y_pred

array([0, 0, 0, ..., 0, 0, 0])